In [ ]:
import numpy as np
import numpy as np
def GMRes_explicit_matrix(A, b, max_iter=None):
    """
    Implementa el algoritmo GMRes para resolver el sistema A x = b.
    
    Args:
        A (ndarray): Matriz del sistema (debe ser cuadrada y explícitamente almacenada).
        b (ndarray): Vector del lado derecho.
        max_iter (int, opcional): Número máximo de iteraciones. Por defecto es el tamaño de A.
    
    Returns:
        x (ndarray): Solución aproximada al sistema A x = b.
    """
    n = A.shape[0]  
    if max_iter is None:
        max_iter = n 

    x = np.zeros(n)  
    r0 = b - A @ x  
    beta = np.linalg.norm(r0) 
    V = np.zeros((n, max_iter + 1))  
    H = np.zeros((max_iter + 1, max_iter)) 
    V[:, 0] = r0 / beta
    for j in range(max_iter):
        w = A @ V[:, j]
        
        for i in range(j + 1):
            H[i, j] = np.dot(V[:, i], w)
            w = w - H[i, j] * V[:, i]
        H[j + 1, j] = np.linalg.norm(w)
        if H[j + 1, j] > 1e-10:
            V[:, j + 1] = w / H[j + 1, j]
        else:
            break  
    e1 = np.zeros(j + 2)
    e1[0] = beta
    y, _, _, _ = np.linalg.lstsq(H[:j + 2, :j + 1], e1, rcond=None)

    x = V[:, :j + 1] @ y

    return x

def compare_bessel(M,J1_1,J1_2pi): 

    def build_D_D2(N):
        D = np.zeros((N,N))
        D2 = np.zeros((N,N))
        j = np.arange(N-1)
        D[j,j+1]=1
        D[j+1,j]=-1
        #Build D2
        i = np.arange(N)
        D2[i,i]=-2
        D2[j,j+1]=1
        D2[j+1,j]=1
        return D, D2
    

    def my_diag(x,n):
        A = np.zeros((n,n))
        i = np.arange(n)
        A[i,i] = x
        return A
    

    D,D2 = build_D_D2(M-1)




    j = np.arange(M+1)
    h = (2*np.pi-1)/M
    h2 = h**2.
    x = 1+j[1:-1]*h
    z0 = J1_1
    zM = J1_2pi
    A_delta = lambda delta: (my_diag(x**2+delta*4*np.sin(x),M-1)/h2)@D2 \
    +(my_diag(x,M-1)/(2*h))@D \
    +my_diag(x**2-1,M-1)
    def b_delta(delta,n):
        b = np.zeros(n)
        b[0] = -((x[0]**2+delta*4*np.sin(x[0]))/h2)*z0+(x[0]/(2*h))*z0
        b[-1] = -((x[-1]**2+delta*4*np.sin(x[-1]))/h2)*zM-(x[-1]/(2*h))*zM
        return b
    A0 = A_delta(0.)
    b0 = b_delta(0.,M-1)
    A1 = A_delta(1.)
    b1 = b_delta(1.,M-1)
    z_delta_0 = np.zeros(M+1)
    z_delta_0[0] = z0
    z_delta_0[-1] = zM
    z_delta_0[1:M] = GMRes_explicit_matrix(A0,b0)
    z_delta_1 = np.zeros(M+1)
    z_delta_1[0] = z0
    z_delta_1[-1] = zM
    z_delta_1[1:M] = GMRes_explicit_matrix(A1,b1)
    d = z_delta_0-z_delta_1
    return d

np.max(compare_bessel(11, 0.44005, -0.21238))


np.float64(0.06847485794900737)

In [63]:
J1_1, J1_2pi =0.44005, -0.21238
def compute_bassel(M,J1_1,J1_2pi):
    j = np.arange(M+1)
    h = (2*np.pi-1)/M
    h2 = h**2.
    x = 1+j[1:-1]*h
    x=np.insert(x,0,1)

    z0 = J1_1
    zM = J1_2pi
    Afun = lambda x,delta: (np.power(x,2)+delta*4*np.sin(x))/h2 #calcula termino diag 
    bfun = lambda x: x/(2*h)

    def my_diag(x,n):
        A = np.zeros((n,n))
        i = np.arange(n)
        A[i,i] = x

        return A
    #DELTA 1
    diag_main_d1= my_diag(-2*Afun(x,1)+np.power(x,2)-1,M)
    subdiag_d1= my_diag(Afun(x,1)-bfun(x),M)
    supdiag_d1= my_diag(Afun(x,1)+bfun(x),M)
    #DELTA 0
    diag_main_d0= my_diag(-2*Afun(x,0)+np.power(x,2)-1,M)
    subdiag_d0= my_diag(Afun(x,0)-bfun(x),M)
    supdiag_d0= my_diag(Afun(x,0)+bfun(x),M)
    
    #Vector b y condiciones iniciales
    A_0 = np.zeros((M,M))
    A_0 +=diag_main_d0+subdiag_d0+supdiag_d0
    A_1 = np.zeros((M,M))
    A_1 +=diag_main_d1+subdiag_d1+supdiag_d1


    b_0 =np.zeros(M)
    b_0[0] = -Afun(x[0], 0) + bfun(x[0])  
    b_0[-1]= -Afun(x[-1],0)+bfun(x[-1])

    print(b_0.shape[0])
    b_1 =np.zeros(M)
    b_1[0]= -Afun(x[0],1)*z0+bfun(x[0])*z0
    b_1[-1]= -Afun(x[-1],1)*zM+bfun(x[-1])*zM

    z_delta_0 = np.zeros(M+1)
    z_delta_0[0] = z0
    z_delta_0[-1] = zM
    z_delta_0[1:] = GMRes_explicit_matrix(A_0,b_0)

    z_delta_1 = np.zeros(M+1)
    z_delta_1[0] = z0
    z_delta_1[-1] = zM
    z_delta_1[1:] = GMRes_explicit_matrix(A_1,b_1)

    d= z_delta_1-z_delta_0
    return d
np.max(compute_bassel(11, 0.44005, -0.21238))


11


np.float64(5.140343485094043)